# ყველა ფაილის მისამართის ამოღება

In [1]:
!pip install librosa
!pip install pydub

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [2]:
def convert_to_wav(files):
    for src in files:
        dst = src[:-3] + "wav"
        s = AudioSegment.from_file(src)
        s.export(dst, format="wav")

In [4]:
folders = os.listdir('data')
for sub_dir in folders:
#     print(sub_dir)
    mp3_files = glob.glob(os.getcwd() + '/data/' + sub_dir + '/*.mp3')
    convert_to_wav(mp3_files)
#     m4a_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.m4a')
#     mp4_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.mp4')
#     aac_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.aac')
#     files = mp3_files + m4a_files + mp4_files + aac_files
#     რატომღაც ამას არ შვება :(
#     convert_to_wav(files) 
#     print(mp3_files)

In [3]:
import pandas as pd
import librosa
import librosa.display

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [4]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
#         zero_crossing_rate = get_zero_crossing_rate(audio)
#         other features
        return [mfccs]
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 

In [17]:
def get_all_features(folder):
    folders = os.listdir(folder)
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/' + folder + '/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'class_label'])
    return data

In [6]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccsscaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, 61), mfccsscaled - (40,)
    return mfccsscaled

In [88]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
#     rate - (1, სხვადასხვა)
    return rate

In [69]:
def get_spectral_centroid(audio):
    #SPECTRAL_CENTROID
    sp = librosa.feature.spectral_centroid(audio)
#     sp - (1, 61)
    print(sp.shape)
    return sp

In [68]:
def get_spectral_roll_ff(audio, sr):
    # Spectral-roll off
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
#     a - (1, 61)
    print(a.shape)
    return a

In [67]:
def get_spectral_flux(audio, sr):
    # Spectral flux
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
#     onset_env - (61,)
    print(onset_env.shape)
    return onset_env

In [66]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, 61)
    print(chroma_stft.shape, chroma_cens.shape, chroma_cq.shape)
    return [chroma_stft, chroma_cens, chroma_cq]

In [65]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, 61)
    print(pitches.shape)
    return pitches

In [18]:
data = get_all_features('data')

In [19]:
data

,mfccs,class_label
0,"[-340.99185, 71.76349, -9.056803, 58.628418, -...",1
1,"[-549.25867, 23.328505, 2.6170156, 16.299675, ...",1
2,"[-576.85846, 15.04707, -0.45508817, 13.05824, ...",1
3,"[-373.90222, 69.62015, 38.59025, 51.163128, 9....",1
4,"[-485.77536, 103.56779, 5.1006126, 12.516183, ...",1
...,...,...
161,"[-428.0815, 65.607445, -6.0010858, 17.54062, -...",5
162,"[-259.54175, 97.03256, -4.835197, 36.80339, -2...",5
163,"[-526.5556, 111.043015, 11.877811, 17.914228, ...",5
164,"[-368.36838, 79.430695, 18.874813, 32.134174, ...",5


In [91]:
labels = [[i] for i in data['class_label']]
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(labels)
print(y_onehot.shape)
y =np.array(labels)

(166, 5)


/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [92]:
X = data.drop(columns=['class_label'])
print(X)
X = X.values
X[0][0].shape
# X
# type(X[0][0])
ls = [] #featurebi gvinda da magitom
for i in range(X.shape[0]):
    ls.append(X[i][0])
ls = np.array(ls)
# ls.shape
# print(ls)

                                                 mfccs
0    [-340.99185, 71.76349, -9.056803, 58.628418, -...
1    [-549.25867, 23.328505, 2.6170156, 16.299675, ...
2    [-576.85846, 15.04707, -0.45508817, 13.05824, ...
3    [-373.90222, 69.62015, 38.59025, 51.163128, 9....
4    [-485.77536, 103.56779, 5.1006126, 12.516183, ...
..                                                 ...
161  [-428.0815, 65.607445, -6.0010858, 17.54062, -...
162  [-259.54175, 97.03256, -4.835197, 36.80339, -2...
163  [-526.5556, 111.043015, 11.877811, 17.914228, ...
164  [-368.36838, 79.430695, 18.874813, 32.134174, ...
165  [-519.13727, 42.85378, -10.486978, 16.277979, ...

[166 rows x 1 columns]


 # ნეირონული ქსელის კოდი

In [93]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [94]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [95]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [96]:
# initial setup
input_size = ls.shape[1]
hidden_size = 25
num_labels = y_onehot.shape[1]
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls.shape[0]
ls = np.matrix(ls)
y = np.matrix(y_onehot)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((25, 41), (5, 26))

In [97]:
a1, z2, a2, z3, h = forward_propagate(ls, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((166, 41), (166, 25), (166, 26), (166, 5), (166, 5))

In [98]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [99]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [100]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls, y_onehot, learning_rate)
J, grad.shape

(3.6494680738721867, (1155,))

In [101]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls, y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 1.8671269695596246
     jac: array([ 1.25929148e-07, -1.15278174e-04,  1.27833705e-04, ...,
       -1.14312358e-03, -1.16075021e-04, -1.42330780e-03])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 40
  status: 3
 success: False
       x: array([ 0.00304011, -0.01895257,  0.02043913, ..., -0.06810234,
       -0.01927369, -0.06581679])

In [102]:
ls = np.matrix(ls)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[1],
       [1],
       [1],
       [1],
       [2],
       [1],
       [2],
       [5],
       [1],
       [2],
       [1],
       [1],
       [1],
       [1],
       [2],
       [1],
       [2],
       [3],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [3],
       [1],
       [1],
       [1],
       [5],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [3],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [3],
       [2],
       [2],
       [2],
       [3],
    

In [22]:
#not done yet
test = get_all_features('test')